# Github repo url

[https://github.com/SkyDocs/Plant-Disease-Detection](https://github.com/SkyDocs/Plant-Disease-Detection)

In [1]:
import os

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor


In [2]:
!unzip dataset.zip

Archive:  dataset.zip
replace dataset/train/Cherry___Powdery_mildew/00b7df55-c789-43d6-a02e-a579ac9d07e6___FREC_Pwd.M 4748.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


dataset classification

In [3]:
class trainData():
    def __inti__(self):
        self.lables, self.image_list = self.load_data()
        
    def load_data(self):
        resize = transforms.Compose([transforms.Resize(256, 256)])
        train_dir = os.listdir(os.path.join("dataset", "train"))
        
        ref = {}
        lables = {}
        image_list = {}
        global_count = 0
         
        for i,dir in enumerate(train_dir):
            ref[dir] = i
            images = os.listdir(os.path.join("dataset", "train", dir))
            count = 0
            for img in images:
                if count < 500:
                    lables[count] = i
                    img = os.path.join("dataset", "train", dir, img)
                    image = Image.open(img)
                    image = ToTensor()(image)
                    image_list[count] = resize(image)
                    count += 1
                    global_count += 1
                    print(i)
                else:
                    pirnt("Taken 500 images for training")
                    break
        
        print(reference)
        return lables,image_list

In [4]:
train_data = trainData()

model

In [5]:
class neuralNet(nn.Module):
    def __init__(self):
        super(neuralNet, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.conv3 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5)
        self.conv4 = nn.Conv2d(in_channels=24, out_channels=48, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=48*12*12,out_features=240)
        self.fc2 = nn.Linear(in_features=240,out_features=120)
        self.out = nn.Linear(in_features=120,out_features=17)
        
    def forward(self, x):
        x = x
        
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = self.conv4(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        
        x = x.reshape(-1, 48*12*12)
        x = x.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.relu(x)
        
        x = self.out(x)
        
        return x

In [6]:
model = neuralNet